In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
%env KERAS_BACKEND = tensorflow
%matplotlib inline 
import matplotlib.pylab as plt 
import tensorflow as tf
import numpy as np 
from keras.preprocessing.image import ImageDataGenerator
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

In [ ]:
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train['labels'] = train['labels'].apply(lambda string: string.split(' '))

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    preprocessing_function = None,
    validation_split= 0.1
)

train_data = datagen.flow_from_dataframe(
    train,
    directory = '../input/resized-plant2021/img_sz_256',
    x_col = 'image',
    y_col = 'labels',
    subset="training",
    color_mode="rgb",
    #target_size = (224,224),
    class_mode="categorical",
    batch_size=16,
    shuffle=False,
    seed=40,
)
valid_data = datagen.flow_from_dataframe(
    train,
    directory = '../input/resized-plant2021/img_sz_256',
    x_col = 'image',
    y_col = 'labels',
    subset="validation",
    color_mode="rgb",
    #target_size = (224,224),
    class_mode="categorical",
    batch_size=16,
    shuffle=False,
    seed=40,
)

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.optimizers import Adam
base_model = InceptionV3(
    include_top=False, weights='../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    )
'''base_model = ResNet50(
    include_top=False, # 是否包含最後的全連接層 (fully-connected layer)
    weights='../input/tf-keras-resnet/resnet50_notop.h5', # None: 權重隨機初始化、'imagenet': 載入預訓練權重
    )'''
from tensorflow.keras.layers import Dense,MaxPooling2D,GlobalAveragePooling2D,BatchNormalization,Activation
x = base_model.output
x = GlobalAveragePooling2D()(x)
#fully connected layer
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
# finally, the softmax for the classifier 
predictions = Dense(6, activation='sigmoid')(x)
model = tf.keras.Model(inputs=base_model.input ,outputs = predictions)
model.compile(optimizer= 'SGD',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
            train_data,  # data from generator
             #steps_per_epoch=1,    # should be number of batches per epoch
            epochs=20,
            validation_data=valid_data,
            steps_per_epoch=train_data.samples//20,
            validation_steps=valid_data.samples//20,
            #validation_steps = 1,
            verbose=True)

In [ ]:
epochs=range(len(history.history['accuracy']))
plt.figure()
plt.plot(epochs,history.history['accuracy'],'b',label='Training acc')
plt.plot(epochs,history.history['val_accuracy'],'r',label='Validation acc')
plt.title('Traing and Validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs,history.history['loss'],'b',label='Training loss')
plt.plot(epochs,history.history['val_loss'],'r',label='Validation val_loss')
plt.title('Traing and Validation loss')
plt.legend()

In [ ]:
from tqdm import tqdm
import PIL

test = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
#將圖片站存至./
for img_name in tqdm(test['image']):
    path = '../input/plant-pathology-2021-fgvc8/test_images/'+str(img_name)
    with PIL.Image.open(path) as img:
        img = img.resize((256,256))
        img.save(f'./{img_name}')

In [ ]:
#讀取./圖片至test_data
test_data = datagen.flow_from_dataframe(
    test,
    directory = './',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    #target_size = (224,224),
    classes=None,
    class_mode=None,
    batch_size=16,
    shuffle=False,
    seed=40,
)
#進行預測
best_threshold = 0.4#
preds = model.predict(test_data)
print(preds)
preds = preds.tolist()

indices = []
for pred in preds:
    temp = []
    for category in pred:
        if category>=best_threshold:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    
print(indices)

In [ ]:
labels = (train_data.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

testlabels = []
for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

In [ ]:
import os
delfiles = tf.io.gfile.glob('./*.jpg')
 
for file in delfiles:
    os.remove(file)
    
sub = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sub['labels'] = testlabels
sub.to_csv('submission.csv', index=False)
sub